In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import os.path
import time
import random
import pickle
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [5]:
# Create negative image library
negative_images = []

for dirpath, dirnames, filenames in os.walk("./faces94"):
    for filename in [f for f in filenames if f.endswith(".jpg")]:
        image_data = plt.imread(os.path.join(dirpath, filename))
        negative_images.append([len(negative_images), 0, image_data])
        
print(len(negative_images), negative_images[1][0], negative_images[1][1], negative_images[0][2].shape)

# Create positive image library
positive_images = []

for filename in os.listdir("./charles"):
    if filename.endswith(".jpg"):
        image_data = plt.imread(os.path.join("./charles", filename))
        positive_images.append([len(negative_images) + len(positive_images), 1, image_data])

print(len(positive_images), positive_images[0][0], positive_images[0][1], positive_images[0][2].shape)

3059 1 0 (200, 180, 3)
71 3059 1 (200, 180, 3)


C:\Users\Charles\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:764: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


In [6]:
# Generate train, validation and test sets
FRAC_TEST = 0.1
FRAC_TRAIN = 0.8

def gen_cross_validation(positive_train_set, negative_train_set, frac_train=0.8):
    """Generate a cross-validation fold."""
    
    n_pos = len(positive_train_set)
    n_neg = len(negative_train_set)
    random.shuffle(positive_train_set)
    random.shuffle(negative_train_set)
    
    train = positive_train_set[:int(frac_train*n_pos)] + negative_train_set[:int(frac_train*n_neg)]
    valid = positive_train_set[int(frac_train*n_pos):] + negative_train_set[int(frac_train*n_neg):]
    random.shuffle(train)
    random.shuffle(valid)
    
    return train, valid
    
# Generate train and test split
n_neg = len(negative_images)
n_pos = len(positive_images)
random.shuffle(positive_images)
random.shuffle(negative_images)

negative_train_set = negative_images[int(FRAC_TEST*n_neg):]
positive_train_set = positive_images[int(FRAC_TEST*n_pos):]
test_set = negative_images[:int(FRAC_TEST*n_neg)] + positive_images[:int(FRAC_TEST*n_pos)]

print(len(negative_train_set), len(positive_train_set), len(test_set), n_neg + n_pos)

2754 64 312 3130


In [7]:
def save_struct(filename, struct):
    file = open(filename, 'wb') 
    pickle.dump(struct, file)
    file.close()

def load_struct(filename):
    file = open(filename, 'rb')  
    struct = pickle.load(file)  
    file.close()
    return struct

# Save the permanent datasets
save_struct('negative_train_set', negative_train_set)
save_struct('positive_train_set', positive_train_set)
save_struct('test_set', test_set)

In [8]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 200, 180, 3])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[5, 5], strides=5)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[4, 4], strides=4)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 10 * 9 * 32])
    dense = tf.layers.dense(inputs=pool2_flat, units=512, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
        inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=2)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"]),
        "precision": tf.metrics.precision(
            labels=labels, predictions=predictions["classes"]),
        "recall": tf.metrics.recall(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [17]:
# Generate a randon cross-validation split
train, valid = gen_cross_validation(positive_train_set, negative_train_set, frac_train=FRAC_TRAIN)
train_data = np.asarray([image for [index, label, image] in train])
train_labels = np.asarray([label for [index, label, image] in train], dtype=np.int32)
eval_data = np.asarray([image for [index, label, image] in valid])
eval_labels = np.asarray([label for [index, label, image] in valid], dtype=np.int32)
print(train_data.shape)
print(train_labels.shape)
print(eval_data.shape)
print(eval_labels.shape)

# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/tmp/convnet_model")

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

(2254, 200, 180, 3)
(2254,)
(564, 200, 180, 3)
(564,)


NameError: name 'tf' is not defined